<a href="https://colab.research.google.com/github/maicken/INF442_P8/blob/master/INF442_P8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data analysis

Now that we have all the data, we will be analysing it

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
# Getting the data
%cd "gdrive/My Drive/Colab Notebooks/INF442_P8"

/content/gdrive/My Drive/Colab Notebooks/INF442_P8


In [11]:
df_c = pd.read_csv('caracteristiques.csv')
df_l = pd.read_csv('lieux.csv')
df_u = pd.read_csv('usagers.csv')
df_v = pd.read_csv('vehicules.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
print("Caracteristiques")
print(df_c.head())
print(df_c.columns)
print()
print("Lieux")
print(df_l.head())
print(df_l.columns)
print()
print("Usagers")
print(df_u.head())
print(df_u.columns)
print()
print("Vehicules")
print(df_v.head())
print(df_v.columns)

Caracteristiques
        Num_Acc  an  mois  jour  ...  gps        lat      long  dep
0  201800000001  18     1    24  ...    M  5055737.0  294992.0  590
1  201800000002  18     2    12  ...    M  5052936.0  293151.0  590
2  201800000003  18     3     4  ...    M  5051243.0  291714.0  590
3  201800000004  18     5     5  ...    M  5051974.0  289123.0  590
4  201800000005  18     6    26  ...    M  5051607.0  290605.0  590

[5 rows x 16 columns]
Index(['Num_Acc', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm',
       'col', 'com', 'adr', 'gps', 'lat', 'long', 'dep'],
      dtype='object')

Lieux
        Num_Acc  catr voie  v1   v2  ...  larrout  surf  infra  situ  env1
0  201800000001     3   41 NaN    C  ...      NaN   1.0    0.0   1.0   0.0
1  201800000002     4   41 NaN    D  ...      NaN   1.0    0.0   1.0   0.0
2  201800000003     3   39 NaN    D  ...      NaN   1.0    0.0   1.0   0.0
3  201800000004     3   39 NaN  NaN  ...      NaN   1.0    0.0   1.0   0.0
4  20180000000

In [13]:
# Information about the columns
print("caracteristiques")
print()
df_c.info()
print()
print("lieux")
print()
df_l.info()
print()
print("usagers")
print()
df_u.info()
print()
print("vehicules")
print()
df_v.info()
print()

caracteristiques

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553424 entries, 0 to 553423
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  553424 non-null  int64  
 1   an       553424 non-null  int64  
 2   mois     553424 non-null  int64  
 3   jour     553424 non-null  int64  
 4   hrmn     553424 non-null  int64  
 5   lum      553424 non-null  int64  
 6   agg      553424 non-null  int64  
 7   int      553424 non-null  int64  
 8   atm      553358 non-null  float64
 9   col      553405 non-null  float64
 10  com      553424 non-null  int64  
 11  adr      482138 non-null  object 
 12  gps      349704 non-null  object 
 13  lat      341505 non-null  float64
 14  long     341501 non-null  float64
 15  dep      553424 non-null  int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 67.6+ MB

lieux

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553424 entries, 0 to 553423
Data columns (total

In [14]:
# Join caracteristiques and lieux into one pandas only
df = df_c.merge(df_l, left_index=True, right_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553424 entries, 0 to 553423
Data columns (total 34 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Num_Acc_x  553424 non-null  int64  
 1   an         553424 non-null  int64  
 2   mois       553424 non-null  int64  
 3   jour       553424 non-null  int64  
 4   hrmn       553424 non-null  int64  
 5   lum        553424 non-null  int64  
 6   agg        553424 non-null  int64  
 7   int        553424 non-null  int64  
 8   atm        553358 non-null  float64
 9   col        553405 non-null  float64
 10  com        553424 non-null  int64  
 11  adr        482138 non-null  object 
 12  gps        349704 non-null  object 
 13  lat        341505 non-null  float64
 14  long       341501 non-null  float64
 15  dep        553424 non-null  int64  
 16  Num_Acc_y  553424 non-null  int64  
 17  catr       553424 non-null  int64  
 18  voie       484820 non-null  object 
 19  v1         2674 non-nul

In [43]:
# Some histogramas to analyse vehicules
print()
print(df_v['senc'].value_counts())
print()
print(df_v['catv'].value_counts())
print()
print(df_v['obs'].value_counts())
print()
print(df_v['obsm'].value_counts())
print()
print(df_v['choc'].value_counts())
print()
print(df_v['manv'].value_counts())


0.0    681840
1.0    159382
2.0    100728
Name: senc, dtype: int64

7     583688
33     77985
10     51847
1      42174
30     40481
2      37307
31     22369
32     21881
34     13542
14      9269
15      8560
37      7163
17      6641
3       3911
13      3632
99      3586
21      1886
38      1871
40      1300
36      1288
20       912
16       400
35       273
39       256
Name: catv, dtype: int64

0.0     819484
1.0      19908
13.0     15165
2.0      13324
4.0      11350
3.0      10419
6.0      10073
8.0       9771
14.0      6660
16.0      5593
12.0      5388
15.0      4523
9.0       3184
11.0      2187
7.0       2122
5.0       1287
10.0      1041
Name: obs, dtype: int64

2.0    635926
0.0    188814
1.0    101842
9.0     11456
6.0      1824
4.0       907
5.0       800
Name: obsm, dtype: int64

1.0    341632
3.0    133996
2.0    108755
4.0     92190
8.0     68541
0.0     64697
7.0     59018
6.0     32166
5.0     25531
9.0     15347
Name: choc, dtype: int64

1.0     402692
2.0     

In [51]:
# Some histogramas to analyse usagers
print()
print(df_u['place'].value_counts())
print()
print(df_u['catu'].value_counts())
print()
print(df_u['grav'].value_counts())
print()
print(df_u['sexe'].value_counts())
print()
print(df_u['an_nais'].value_counts())
print()
print(df_u['secu'].value_counts())
print()
print(df_u['locp'].value_counts())
print()
print(df_u['actp'].value_counts())
print()
print(df_u['etatp'].value_counts())
print()


1.0    917361
2.0    128505
3.0     28353
4.0     24023
5.0     12418
7.0      3654
8.0      3640
9.0      3546
6.0      1237
Name: place, dtype: int64

1    917305
2    205432
3    107376
4      2120
Name: catu, dtype: int64

1    504629
4    442501
3    251731
2     33372
Name: grav, dtype: int64

1    831047
2    401186
Name: sexe, dtype: int64

1990.0    33560
1991.0    33005
1992.0    32383
1989.0    31776
1988.0    31259
          ...  
1912.0        5
1909.0        2
1908.0        2
1907.0        2
1906.0        2
Name: an_nais, Length: 114, dtype: int64

11.0    700522
21.0    223718
13.0     91236
93.0     74863
23.0     22173
12.0     18600
22.0     13008
31.0      8774
3.0       7691
92.0      4931
91.0      4806
1.0       2729
2.0       2588
41.0      1703
42.0      1390
43.0      1091
33.0       464
32.0       419
Name: secu, dtype: int64

0.0    1077498
2.0      25370
4.0      24697
3.0      23955
1.0      14865
5.0       6682
6.0       2147
8.0       1134
7.0        117

# Getting data and saving them at my Google Drive folder

In [1]:
# web scraping

!pip install selenium
!pip install beautifulsoup4
!pip install wget

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import wget
import traceback

In [3]:
url = 'https://www.data.gouv.fr/fr/datasets/base-de-donnees-accidents-corporels-de-la-circulation/#_'

year = 2010 # First year

response = requests.get(url, timeout=10)
if response.status_code == 200:
  print('Permission to access granted')
  print()
soup = BeautifulSoup(response.content, 'html.parser')
# get list of names
list_names = []
# Selecting all article type
for file in soup.findAll('article', attrs={'class':'card resource-card '}):
  # Get the name of the file
  name = file.findAll('h4', attrs={'class':'ellipsis'})[0].get_text()
  # Check if its a file
  if name.endswith('.csv') and int(name[-8:-4]) >= year:
    # Get the file
    list_names.append(name)
    filename = file.findAll('a', attrs={'class':'btn btn-sm btn-primary'})[1]['href']
    filedata = wget.download(filename, 'sample_data/' + name)

Permission to access granted



In [7]:
v = []
u = []
l = []
c = []

def sort_names(name):
  return(name[:-10]+name[-8:])
list_names_s = sorted(list_names, key=sort_names)

n = len(list_names_s) / 4
for i, name in enumerate(list_names_s, 0):
  if 0 <= i < n:
    c.append(name)
  elif n <= i < 2*n:
    l.append(name)
  elif 2*n <= i < 3*n:
    u.append(name)
  else:
    v.append(name)

df_c = pd.DataFrame()
df_l = pd.DataFrame()
df_u = pd.DataFrame()
df_v = pd.DataFrame()

print("--------------------")
print("caracteristiques.csv")
print("--------------------")
er = []
for file in c:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_c = pd.concat([data, df_c], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_c))
    print("num_columns = %d" %len(df_c.columns))
print()
print('Error in years :')
print(er)
print()
print("--------------------")
print("lieux.csv")
print("--------------------")
er = []
for file in l:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_l = pd.concat([data, df_l], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_l))
    print("num_columns = %d" %len(df_l.columns))
print()
print('Error in years :')
print(er)
print()
print("--------------------")
print("usagers.csv")
print("--------------------")
er = []
for file in u:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_u = pd.concat([data, df_u], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_u))
    print("num_columns = %d" %len(df_u.columns))
print()
print('Error in years :')
print(er)
print()
print("--------------------")
print("vehicules.csv")
print("--------------------")
er = []
for file in v:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_v = pd.concat([data, df_v], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_v))
    print("num_columns = %d" %len(df_v.columns))
print()
print('Error in years :')
print(er)
print()
df_c.to_csv('sample_data/caracteristiques.csv')
df_l.to_csv('sample_data/lieux.csv')
df_u.to_csv('sample_data/usagers.csv')
df_v.to_csv('sample_data/vehicules.csv')

--------------------
caracteristiques.csv
--------------------

size = 69379
num_columns = 15

size = 136353
num_columns = 15

size = 198603
num_columns = 15

size = 257000
num_columns = 15

size = 316854
num_columns = 15

size = 375508
num_columns = 15

size = 434940
num_columns = 15

size = 495641
num_columns = 15

size = 553424
num_columns = 15

Error in years :
[]

--------------------
lieux.csv
--------------------

size = 69379
num_columns = 17

size = 136353
num_columns = 17

size = 198603
num_columns = 17

size = 257000
num_columns = 17

size = 316854
num_columns = 17

size = 375508
num_columns = 17

size = 434940
num_columns = 17

size = 495641
num_columns = 17

size = 553424
num_columns = 17

Error in years :
[]

--------------------
usagers.csv
--------------------

size = 154192
num_columns = 11

size = 302735
num_columns = 11

size = 441363
num_columns = 11

size = 570057
num_columns = 11

size = 702243
num_columns = 11

size = 832621
num_columns = 11

size = 966043
num_co

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Pass data to my google drive
%mv "sample_data/lieux.csv" "drive/My Drive/Colab Notebooks/INF442_P8/lieux.csv"
%mv "sample_data/caracteristiques.csv" "drive/My Drive/Colab Notebooks/INF442_P8/caracteristiques.csv"
%mv "sample_data/usagers.csv" "drive/My Drive/Colab Notebooks/INF442_P8/usagers.csv"
%mv "sample_data/vehicules.csv" "drive/My Drive/Colab Notebooks/INF442_P8/vehicules.csv"

%cd "drive/My Drive/Colab Notebooks/INF442_P8/"

/content/drive/My Drive/Colab Notebooks/INF442_P8
